In [ ]:
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langchain_teddynote.tools.tavily import TavilySearch

from langchain_teddynote.graphs import visualize_graph
from langgraph.prebuilt import ToolNode, tools_condition



## 도구 사용 분기 처리

1. state 생성
2. llm 생성
3. 도구 리스트 생성
4. llm과 도구 연결
5. 노드 생성
6. 노드 추가
7. 노드간 엣지 추가
8. 분기 조건처리
9. 그래프 확인

In [ ]:

# 1. state
class State(TypedDict):
    messages : Annotated[list,add_messages]

# 2. llm 
llm = ChatOpenAI(model="gpt-4.1-mini",temperature=0)

# 3. 도구 생성
tool = TavilySearch(max_results=3)
tools = [tool]
tool_node = ToolNode(tools)

In [ ]:

# 4. llm 과 구 연결
llm_bind_tool = llm.bind_tools(tools)

# 5. 노드 생성
def chatbot(state:State):
    # print(state[('messages')])
    return State({'messages':[llm_bind_tool.invoke(state['messages'])]})

# 8. 분기 처리
def route_tool_use(state:State):
    if messages := state.get("messages", []):
        # 가장 최근 AI 메시지 추출
        ai_message = messages[-1]
    else:
        # 입력 상태에 메시지가 없는 경우 예외 발생
        raise ValueError(f"No messages found in input state to tool_edge: {state}")

    # AI 메시지에 도구 호출이 있는 경우 "tools" 반환
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        # 도구 호출이 있는 경우 "tools" 반환
        return "tool_node"
    # 도구 호출이 없는 경우 "END" 반환
    return END
    

# 6. 노드 추가
state_graph = StateGraph(State)
state_graph.add_node('chatbot',chatbot)
state_graph.add_node('tool_node',tool_node)
# 7. 엣지 추가
state_graph.add_edge(START,'chatbot')
# 8. 분기 처리
state_graph.add_conditional_edges(
    source='chatbot',
    path=route_tool_use,
    path_map={
        'tool_node':'tool_node',
        END:END
    }
)
# 위에랑 같은 처리 route_tool_use 직접 구현없이 tools_condition으로 구현가능
# state_graph.add_conditional_edges(
#     source='chatbot',
#     path=tools_condition
# )
state_graph.add_edge('tool_node','chatbot')
state_graph.add_edge('chatbot',END)


In [ ]:

graph = state_graph.compile()


# 그래프 시각화
visualize_graph(graph)

In [ ]:
graph.invoke({"messages":"오늘의뉴스"})